In [2]:
import numpy as np

# Create Board and Display

In [3]:
def create_board():
    board = np.full((9, 9), None)
    cells_per_row = [5, 6, 7, 8, 9, 8, 7, 6, 5]
    
    # D'abord on remplit toutes les cases valides avec 0
    for y, n_cells in enumerate(cells_per_row):
        for x in range(n_cells):
            board[y][x] = 0
    
    # Placement des billes blanches (en haut)
    for x in range(5):  # Ligne 0
        board[0][x] = 1
    for x in range(6):  # Ligne 1
        board[1][x] = 1
    # Ligne 2 : 2 espaces, 3 blanches, 2 espaces
    for x in range(2, 5):
        board[2][x] = 1
        
    # Placement des billes noires (en bas)
    for x in range(5):  # Ligne 8
        board[8][x] = -1
    for x in range(6):  # Ligne 7
        board[7][x] = -1
    # Ligne 6 : 2 espaces, 3 noires, 2 espaces
    for x in range(2, 5):
        board[6][x] = -1
    
    return board


In [4]:
def display_board(board):
    cells_per_row = [5, 6, 7, 8, 9, 8, 7, 6, 5]
    
    for y, n_cells in enumerate(cells_per_row):
        # Calculer l'indentation pour cette ligne
        # Les premières lignes ont plus d'indentation, puis ça diminue jusqu'à la ligne du milieu
        indent = abs(4 - y)  # 4 espaces pour la première ligne, 0 pour la ligne du milieu
        
        # Afficher l'indentation
        print(" " * indent, end="")
        
        # Afficher les cases de la ligne
        for x in range(n_cells):
            if board[y][x] == 0:
                print("O ", end="")  # Case vide
            elif board[y][x] == 1:
                print("W ", end="")  # Bille blanche (white)
            elif board[y][x] == -1:
                print("B ", end="")  # Bille noire (black)
        print()  # Nouvelle ligne


In [5]:
board = create_board()
print("Position initiale:")
display_board(board)

Position initiale:
    W W W W W 
   W W W W W W 
  O O W W W O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O B B B O O 
   B B B B B B 
    B B B B B 


# Get cell content

In [6]:
def get_cell_content(board, x, y):
    # D'abord vérifier si les coordonnées sont dans les limites du plateau
    if x < 0 or y < 0 or x > 8 or y > 8:
        return "Invalid coordinates: out of bounds"
    
    # Vérifier si la case existe pour cette ligne (selon le nombre de cases par ligne)
    cells_per_row = [5, 6, 7, 8, 9, 8, 7, 6, 5]
    if x >= cells_per_row[y]:
        return "Invalid coordinates: cell doesn't exist"
    
    # Si on arrive ici, les coordonnées sont valides
    # Convertir la valeur numérique en symbole
    if board[y][x] == 1:
        return "W"
    elif board[y][x] == -1:
        return "B"
    elif board[y][x] == 0:
        return "O"
    else:
        return "Invalid cell"

# Get neighbor

In [7]:
def get_neighbors(board, x, y):
    neighbors = {}
    
    # Partie haute du plateau (lignes 0-4)
    if y < 4:
        directions = {
            'NW': (x-1, y-1),
            'NE': (x, y-1),
            'E': (x+1, y),
            'W': (x-1, y),
            'SW': (x, y+1),
            'SE': (x+1, y+1)
        }
    elif y==4:
        directions = {
            'NW': (x-1, y-1),
            'NE': (x, y-1),
            'E': (x+1, y),
            'W': (x-1, y),
            'SW': (x-1, y+1),
            'SE': (x, y+1)
        }
        # Partie basse du plateau (lignes 5-8)

    else:
        directions = {
            'NW': (x, y-1),    # +1 compensé pour le nord
            'NE': (x+1, y-1),  # +1 compensé pour le nord
            'E': (x+1, y),
            'W': (x-1, y),
            'SW': (x-1, y+1),  # -1 compensé pour le sud
            'SE': (x, y+1)     # -1 compensé pour le sud
        }
    
    # On vérifie quels voisins sont valides
    for direction, (nx, ny) in directions.items():
        content = get_cell_content(board, nx, ny)
        if content in ['W', 'B', 'O']:  # Si c'est une case valide
            neighbors[direction] = (nx, ny)
            
    return neighbors


# Move one 

In [8]:
def move_piece(board, start_x, start_y, end_x, end_y):
    # Vérifier si la case de départ contient une bille
    start_content = get_cell_content(board, start_x, start_y)
    if start_content not in ['W', 'B']:
        return False, "Pas de bille à déplacer sur la case de départ"
    
    # Vérifier si la case d'arrivée est valide et vide
    end_content = get_cell_content(board, end_x, end_y)
    if end_content not in ['O']:
        return False, "La case d'arrivée n'est pas valide ou n'est pas vide"
    
    # Vérifier si la case d'arrivée est un voisin valide
    neighbors = get_neighbors(board, start_x, start_y)
    destination = None
    for direction, (nx, ny) in neighbors.items():
        if nx == end_x and ny == end_y:
            destination = (nx, ny)
            break
    
    if destination is None:
        return False, "La case d'arrivée n'est pas un voisin valide"
    
    # Si toutes les conditions sont remplies, effectuer le déplacement
    board[end_y][end_x] = board[start_y][start_x]  # Déplacer la bille
    board[start_y][start_x] = 0  # Vider la case de départ
    
    return True, "Déplacement effectué avec succès"



# Valids groups

In [9]:
def is_valid_group(board, coordinates):
    """
    Vérifie si un groupe de coordonnées forme un groupe valide
    Args:
        board: le plateau de jeu
        coordinates: liste de tuples (x,y) représentant les positions des billes
    Returns:
        (bool, str, str): (est_valide, message, direction_alignement)
    """
    # Vérifier le nombre de billes
    if len(coordinates) < 2 or len(coordinates) > 3:
        return False, "Le groupe doit contenir 2 ou 3 billes", None
    
    # Vérifier que toutes les positions contiennent des billes de même couleur
    first_x, first_y = coordinates[0]
    color = get_cell_content(board, first_x, first_y)
    if color not in ['W', 'B']:
        return False, "La première position ne contient pas de bille", None
    
    for x, y in coordinates[1:]:
        if get_cell_content(board, x, y) != color:
            return False, "Les billes ne sont pas de la même couleur", None
    
    # Vérifier que les billes sont adjacentes et alignées
    # Pour chaque bille, on vérifie qu'elle a au moins une autre bille du groupe comme voisine
    alignment_direction = None
    for i, (x, y) in enumerate(coordinates):
        neighbors = get_neighbors(board, x, y)
        has_neighbor = False
        
        # Pour chaque autre bille du groupe
        for j, (other_x, other_y) in enumerate(coordinates):
            if i != j:  # Ne pas se comparer soi-même
                # Vérifier si cette bille est un voisin
                for direction, (nx, ny) in neighbors.items():
                    if nx == other_x and ny == other_y:
                        if alignment_direction is None:
                            alignment_direction = direction
                        elif alignment_direction != direction:
                            # Vérifier si la direction est opposée (ex: NW-SE, E-W, NE-SW)
                            if not (
                                (alignment_direction == 'NW' and direction == 'SE') or
                                (alignment_direction == 'SE' and direction == 'NW') or
                                (alignment_direction == 'E' and direction == 'W') or
                                (alignment_direction == 'W' and direction == 'E') or
                                (alignment_direction == 'NE' and direction == 'SW') or
                                (alignment_direction == 'SW' and direction == 'NE')
                            ):
                                return False, "Les billes ne sont pas alignées", None
                        has_neighbor = True
                        break
        
        if not has_neighbor:
            return False, "Les billes ne sont pas toutes adjacentes", None
    
    return True, "Groupe valide", alignment_direction




# Group movement

## Parallel movement

In [10]:
def move_group_parallel(board, coordinates, direction):
    """
    Déplace un groupe de billes parallèlement à leur alignement
    """
    print("ICCCCCCCCCCCCCCCIIIIIIIIIIIIIIIIIII")
    # Vérifier si le groupe est valide
    is_valid, message, alignment = is_valid_group(board, coordinates)
    if not is_valid:
        print("not valid")
        return False, message
        
    # Calculer les nouvelles coordonnées selon la direction
    destinations = []
    first_y = coordinates[0][1]
    print("first_y:",first_y)
    
    # Appliquer les modifications de coordonnées selon la partie du plateau
    # et la direction (comme dans get_neighbors)
    if first_y < 4:  # Partie haute
        print("partie haute")
        for x, y in coordinates:
            if direction == 'NW':
                destinations.append((x-1, y-1))
            elif direction == 'NE':
                destinations.append((x, y-1))
            elif direction == 'E':
                destinations.append((x+1, y))
            elif direction == 'W':
                destinations.append((x-1, y))
            elif direction == 'SW':
                destinations.append((x, y+1))
            elif direction == 'SE':
                destinations.append((x+1, y+1))
    elif first_y == 4:  # Ligne du milieu
        print("milieu")
        for x, y in coordinates:
            if direction == 'NW':
                destinations.append((x-1, y-1))
            elif direction == 'NE':
                destinations.append((x, y-1))
            elif direction == 'E':
                destinations.append((x+1, y))
            elif direction == 'W':
                destinations.append((x-1, y))
            elif direction == 'SW':
                destinations.append((x-1, y+1))
            elif direction == 'SE':
                destinations.append((x, y+1))
    else:  # Partie basse
        print("basse")
        for x, y in coordinates:
            if direction == 'NW':
                destinations.append((x, y-1))
            elif direction == 'NE':
                destinations.append((x+1, y-1))
            elif direction == 'E':
                destinations.append((x+1, y))
            elif direction == 'W':
                destinations.append((x-1, y))
            elif direction == 'SW':
                destinations.append((x-1, y+1))
            elif direction == 'SE':
                destinations.append((x, y+1))
    
    # Vérifier que toutes les destinations sont valides
    print("destinations",destinations)
    for dest_x, dest_y in destinations:
        content = get_cell_content(board, dest_x, dest_y)
        if content not in ['O']:
            print("Une ou plusieurs cases de destination sont occupées ou invalides")
            return False, "Une ou plusieurs cases de destination sont occupées ou invalides"
    
    # Effectuer le déplacement
    print("destinations",destinations)
    piece_color = board[coordinates[0][1]][coordinates[0][0]]
    
    # D'abord vider toutes les positions de départ
    for x, y in coordinates:
        board[y][x] = 0
    
    # Puis remplir les destinations
    for dest_x, dest_y in destinations:
        board[dest_y][dest_x] = piece_color
    
    return True, "Mouvement parallèle effectué avec succès"

In [11]:
def move_group_parallel(board, coordinates, direction):
   """
   Déplace un groupe de billes parallèlement à leur alignement
   """
   # Vérifier si le groupe est valide
   is_valid, message, alignment = is_valid_group(board, coordinates)
   if not is_valid:
       return False, message
       
   # Calculer les nouvelles coordonnées selon la direction
   destinations = []
   
   # Calculer les destinations pour chaque bille selon sa position sur le plateau
   for x, y in coordinates:
       if y < 4:  # Partie haute
           if direction == 'NW':
               destinations.append((x-1, y-1))
           elif direction == 'NE':
               destinations.append((x, y-1))
           elif direction == 'E':
               destinations.append((x+1, y))
           elif direction == 'W':
               destinations.append((x-1, y))
           elif direction == 'SW':
               destinations.append((x, y+1))
           elif direction == 'SE':
               destinations.append((x+1, y+1))
       elif y == 4:  # Ligne du milieu
           if direction == 'NW':
               destinations.append((x-1, y-1))
           elif direction == 'NE':
               destinations.append((x, y-1))
           elif direction == 'E':
               destinations.append((x+1, y))
           elif direction == 'W':
               destinations.append((x-1, y))
           elif direction == 'SW':
               destinations.append((x-1, y+1))
           elif direction == 'SE':
               destinations.append((x, y+1))
       else:  # Partie basse
           if direction == 'NW':
               destinations.append((x, y-1))
           elif direction == 'NE':
               destinations.append((x+1, y-1))
           elif direction == 'E':
               destinations.append((x+1, y))
           elif direction == 'W':
               destinations.append((x-1, y))
           elif direction == 'SW':
               destinations.append((x-1, y+1))
           elif direction == 'SE':
               destinations.append((x, y+1))
   
   # Vérifier que toutes les destinations sont valides
   for dest_x, dest_y in destinations:
       content = get_cell_content(board, dest_x, dest_y)
       if content not in ['O']:
           return False, "Une ou plusieurs cases de destination sont occupées ou invalides"
   
   # Effectuer le déplacement
   piece_color = board[coordinates[0][1]][coordinates[0][0]]
   
   # D'abord vider toutes les positions de départ
   for x, y in coordinates:
       board[y][x] = 0
   
   # Puis remplir les destinations
   for dest_x, dest_y in destinations:
       board[dest_y][dest_x] = piece_color
   
   return True, "Mouvement parallèle effectué avec succès"

## Inline movement

In [12]:
# def move_group_inline(board, coordinates, direction):
#     """
#     Déplace un groupe de billes dans la direction de leur alignement
#     """
#     # Vérifier si le groupe est valide
#     is_valid, message, alignment = is_valid_group(board, coordinates)
#     if not is_valid:
#         return False, message
    
#     # Récupérer la ligne (y) de la première bille pour savoir si on est dans la partie haute ou basse
#     y_pos = coordinates[0][1]
#     is_lower_half = y_pos >= 4
    
#     # Si les billes sont sur la même colonne (même x)
#     if len(set(x for x, y in coordinates)) == 1:
#         if is_lower_half:
#             print("EHHHHHHHHHHHHHHHHHHH1")
#             if direction != 'NW':  # Dans la partie basse, seul NW est valide pour une ligne verticale
#                 print("Dans la partie basse, seul le mouvement NW est valide pour un groupe vertical")
#                 return False, "Dans la partie basse, seul le mouvement NW est valide pour un groupe vertical"
#         else:
#             print("EHHHHHHHHHHHHHHHHHHH2")
#             if direction != 'SW':  # Dans la partie haute, seul SW est valide pour une ligne verticale
#                 return False, "Dans la partie haute, seul le mouvement SW est valide pour un groupe vertical"
    
#     # Si les billes sont sur la même ligne (même y)
#     elif len(set(y for x, y in coordinates)) == 1:
#         print("EHHHHHHHHHHHHHHHHHHH3")
#         if direction not in ['E', 'W']:
#             return False, "Direction invalide pour un groupe aligné horizontalement"
    
#     # Pour les alignements diagonaux, la vérification dépend aussi de la partie du plateau
#     else:
#         print("EHHHHHHHHHHHHHHHHHHH5")
#         if is_lower_half:
#             # Logique spécifique pour la partie basse
#             pass  # À compléter si nécessaire
#         else:
#             # Logique spécifique pour la partie haute
#             pass  # À compléter si nécessaire
    
#     # Le reste du code pour le déplacement...
#     if direction in ['E']:
#         sorted_coordinates = sorted(coordinates, key=lambda pos: pos[0], reverse=True)
#     elif direction in ['W']:
#         sorted_coordinates = sorted(coordinates, key=lambda pos: pos[0])
#     elif direction in ['SE', 'SW', 'NW', 'NE']:
#         print("EHHHHHHHHHHHHHHHHHHH6")
#         # Pour les mouvements diagonaux, on trie selon y
#         if direction in ['SE', 'SW']:
#             print("EHHHHHHHHHHHHHHHHHHH7")
#             sorted_coordinates = sorted(coordinates, key=lambda pos: pos[1], reverse=True)
#         else:  # NW, NE
#             print("EHHHHHHHHHHHHHHHHHHH8")
#             sorted_coordinates = sorted(coordinates, key=lambda pos: pos[1])
    
#     lead_x, lead_y = sorted_coordinates[0]
#     neighbors = get_neighbors(board, lead_x, lead_y)
    
#     if direction not in neighbors:
#         print("EHHHHHHHHHHHHHHHHHHH9")
#         return False, "Mouvement impossible : destination hors plateau"
    
#     dest_x, dest_y = neighbors[direction]
#     if get_cell_content(board, dest_x, dest_y) != 'O':
#         print("EHHHHHHHHHHHHHHHHHHH10")
#         return False, "La case de destination n'est pas libre"
    
#     piece_color = board[lead_y][lead_x]
    
#     # Effectuer le déplacement
#     for x, y in coordinates:
#         board[y][x] = 0
        
#     board[dest_y][dest_x] = piece_color
    
#     for i in range(len(sorted_coordinates)-1):
#         next_x, next_y = sorted_coordinates[i]
#         board[next_y][next_x] = piece_color

#     return True, "Mouvement en ligne effectué avec succès"



## Inline clean

In [13]:
def move_group_inline(board, coordinates, direction):
   """
   Déplace un groupe de billes dans la direction de leur alignement
   Args:
       board: le plateau de jeu
       coordinates: liste de tuples (x,y) des positions des billes
       direction: direction du mouvement ('NW', 'NE', 'E', 'SE', 'SW', 'W')
   Returns:
       (bool, str): (succès, message)
   """
   # Vérifier si le groupe est valide
   is_valid, message, alignment = is_valid_group(board, coordinates)
   if not is_valid:
       return False, message
   
   # Vérifier la validité du mouvement selon l'alignement
   # Si les billes sont sur la même colonne (même x)
   if len(set(x for x, y in coordinates)) == 1:
       if direction not in ['NW', 'SE'] and direction not in ['NE', 'SW']:
           print("Direction invalide pour un groupe vertical : doit être NW/SE ou NE/SW")
           return False, "Direction invalide pour un groupe vertical : doit être NW/SE ou NE/SW"
   
   # Si les billes sont sur la même ligne (même y)
   elif len(set(y for x, y in coordinates)) == 1:
       if direction not in ['E', 'W']:
           print("Direction invalide pour un groupe aligné horizontalement")
           return False, "Direction invalide pour un groupe aligné horizontalement"
   
   # Si les billes sont alignées diagonalement
   else:
       if alignment == 'NE' and direction not in ['NE', 'SW']:
           print("Direction invalide pour un groupe aligné en NE/SW")
           return False, "Direction invalide pour un groupe aligné en NE/SW"
       elif alignment == 'SE' and direction not in ['SE', 'NW']:
           print("Direction invalide pour un groupe aligné en SE/NW")
           return False, "Direction invalide pour un groupe aligné en SE/NW"
   
   # Trier les coordonnées selon la direction du mouvement
   if direction in ['E']:
       sorted_coordinates = sorted(coordinates, key=lambda pos: pos[0], reverse=True)
   elif direction in ['W']:
       sorted_coordinates = sorted(coordinates, key=lambda pos: pos[0])
   elif direction in ['SE', 'SW', 'NW', 'NE']:
       # Pour les mouvements diagonaux, on trie selon y
       if direction in ['SE', 'SW']:
           sorted_coordinates = sorted(coordinates, key=lambda pos: pos[1], reverse=True)
       else:  # NW, NE
           sorted_coordinates = sorted(coordinates, key=lambda pos: pos[1])
   
   # Vérifier si le mouvement est possible
   lead_x, lead_y = sorted_coordinates[0]
   neighbors = get_neighbors(board, lead_x, lead_y)
   
   if direction not in neighbors:
       return False, "Mouvement impossible : destination hors plateau"
   
   dest_x, dest_y = neighbors[direction]
   if get_cell_content(board, dest_x, dest_y) != 'O':
       return False, "La case de destination n'est pas libre"
   
   # Effectuer le déplacement
   piece_color = board[lead_y][lead_x]
   
   # Vider les positions initiales
   for x, y in coordinates:
       board[y][x] = 0
   
   # Placer les billes dans leurs nouvelles positions
   board[dest_y][dest_x] = piece_color
   
   for i in range(len(sorted_coordinates)-1):
       next_x, next_y = sorted_coordinates[i]
       board[next_y][next_x] = piece_color

   return True, "Mouvement en ligne effectué avec succès"

## All together

In [14]:
def make_move(board, coordinates, direction):
    """
    Fonction générale pour effectuer un mouvement
    Args:
        board: le plateau de jeu
        coordinates: liste de tuples (x,y) des positions des billes à déplacer
        direction: direction du mouvement ('NW', 'NE', 'E', 'SE', 'SW', 'W')
    Returns:
        (bool, str): (succès, message)
    """
    # Vérifier si les coordonnées sont valides
    for x, y in coordinates:
        content = get_cell_content(board, x, y)
        if content not in ['W', 'B']:
            return False, f"Pas de bille en position ({x},{y})"
    
    # Cas d'une seule bille
    if len(coordinates) == 1:
        x, y = coordinates[0]
        return move_piece(board, x, y, *get_neighbors(board, x, y)[direction])
    
    # Cas d'un groupe (2 ou 3 billes)
    elif 2 <= len(coordinates) <= 3:
        is_valid, message, alignment = is_valid_group(board, coordinates)
        if not is_valid:
            return False, message
        
        # Standardiser l'alignement
        if alignment == 'W':
            alignment = 'E'
        elif alignment == 'SW':
            alignment = 'NE'
        elif alignment == 'NW':
            alignment = 'SE'
        
        # Vérifier si c'est un mouvement inline
        is_inline = False
        # if alignment == 'E' and direction in ['E', 'W']:
        #     is_inline = True
        # elif alignment == 'NE' and direction in ['NE', 'SW']:
        #     is_inline = True
        # elif alignment == 'SE' and direction in ['SE', 'NW']:
        #     is_inline = True
        
        #new
        if alignment == 'E' and direction in ['E', 'W']:
            is_inline = True
        elif alignment == 'NE' and direction in ['NE', 'SW']:  # Changé
            is_inline = True
        elif alignment == 'SE' and direction in ['SE', 'NW']:  # Changé
            is_inline = True
        # import pdb;pdb.set_trace()
        if is_inline:
            return move_group_inline(board, coordinates, direction)
        else:
            return move_group_parallel(board, coordinates, direction)
    
    else:
        return False, "Nombre invalide de billes sélectionnées"

# Valid push

In [15]:
def is_valid_push(board, pushing_coordinates, direction):
    print(f"\nDébut vérification poussée:")
    print(f"Coordonnées initiales: {pushing_coordinates}")
    
    # Tri des coordonnées dans l'ordre de la poussée
    # Pour E, on veut la plus à gauche en premier
    if direction == 'E':
        pushing_coordinates = sorted(pushing_coordinates, key=lambda pos: pos[0])
    elif direction == 'W':
        pushing_coordinates = sorted(pushing_coordinates, key=lambda pos: pos[0], reverse=True)
    elif direction in ['SE', 'SW']:
        pushing_coordinates = sorted(pushing_coordinates, key=lambda pos: pos[1])
    else:  # NW, NE
        pushing_coordinates = sorted(pushing_coordinates, key=lambda pos: pos[1], reverse=True)
    
    print(f"Coordonnées après tri: {pushing_coordinates}")
    
    # Vérification du groupe
    is_valid, message, alignment = is_valid_group(board, pushing_coordinates)
    print(f"Vérification groupe: valide={is_valid}, message={message}, alignment={alignment}")
    if not is_valid:
        return False, f"Groupe de billes qui poussent invalide: {message}", []
    
    # Récupérer la couleur des billes qui poussent
    pushing_color = get_cell_content(board, pushing_coordinates[0][0], pushing_coordinates[0][1])
    
    # Vérifier que toutes les positions entre la première et la dernière bille sont occupées par nos billes
    for i in range(len(pushing_coordinates)-1):
        current = pushing_coordinates[i]
        next_pos = pushing_coordinates[i+1]
        neighbors = get_neighbors(board, current[0], current[1])
        if direction not in neighbors or neighbors[direction] != next_pos:
            return False, "Les billes qui poussent ne sont pas adjacentes dans la direction de poussée", []
    
    # Trouver la position après la dernière bille qui pousse
    last_x, last_y = pushing_coordinates[-1]
    neighbors = get_neighbors(board, last_x, last_y)
    if direction not in neighbors:
        return False, "Pas de case valide dans cette direction", []
    
    next_x, next_y = neighbors[direction]
    print(f"Position à pousser: ({next_x},{next_y})")
    
    # Identifier les billes qui vont être poussées
    pushed_coordinates = []
    current_x, current_y = next_x, next_y
    
    while True:
        content = get_cell_content(board, current_x, current_y)
        print(f"Vérification position ({current_x},{current_y}): contenu={content}")
        
        if content not in ['W', 'B']:  # Case vide ou invalide
            break
        if content == pushing_color:  # Bille amie
            return False, "Bille amie bloque la poussée", []
            
        pushed_coordinates.append((current_x, current_y))
        
        neighbors = get_neighbors(board, current_x, current_y)
        if direction not in neighbors:
            break
        current_x, current_y = neighbors[direction]
    
    print(f"Billes poussées trouvées: {pushed_coordinates}")
    
    # Vérifications finales
    if len(pushed_coordinates) == 0:
        return False, "Pas de billes à pousser", []
        
    if len(pushed_coordinates) >= len(pushing_coordinates):
        return False, "Pas assez de billes pour pousser", []
        
    # if len(pushing_coordinates) + len(pushed_coordinates) > 3:
    #     return False, "Trop de billes impliquées dans la poussée", []
    
    return True, "Poussée valide", pushed_coordinates

# Push Marbles

In [16]:
# def push_marbles(board, pushing_coordinates, direction):
#     """
#     Effectue une poussée de billes si elle est valide
#     Args:
#         board: le plateau de jeu
#         pushing_coordinates: liste de tuples (x,y) des billes qui poussent
#         direction: direction de la poussée
#     Returns:
#         (bool, str, int): (succès, message, nombre_billes_sorties)
#     """
#     # Vérifier d'abord si la poussée est valide
#     is_valid, message, pushed_coordinates = is_valid_push(board, pushing_coordinates, direction)
#     if not is_valid:
#         print("Not a push")
#         return False, message, 0
    
#     # Sauvegarder les couleurs
#     pushing_color = board[pushing_coordinates[0][1]][pushing_coordinates[0][0]]
#     if pushed_coordinates:
#         pushed_color = board[pushed_coordinates[0][1]][pushed_coordinates[0][0]]
    
#     # Compter combien de billes vont sortir du plateau
#     billes_sorties = 0
#     for x, y in pushed_coordinates:
#         neighbors = get_neighbors(board, x, y)
#         if direction not in neighbors:
#             billes_sorties += 1
    
#     # Déplacer d'abord les billes poussées
#     # On commence par la dernière bille pour éviter les conflits
#     for x, y in reversed(pushed_coordinates):
#         neighbors = get_neighbors(board, x, y)
#         board[y][x] = 0  # Vider la position actuelle
        
#         # Si la bille ne sort pas du plateau, la déplacer
#         if direction in neighbors:
#             next_x, next_y = neighbors[direction]
#             board[next_y][next_x] = pushed_color
    
#     # Ensuite déplacer les billes qui poussent
#     # Là aussi, on commence par la dernière
#     for i in range(len(pushing_coordinates)-1, -1, -1):
#         x, y = pushing_coordinates[i]
#         neighbors = get_neighbors(board, x, y)
#         next_x, next_y = neighbors[direction]
        
#         board[y][x] = 0  # Vider la position actuelle
#         board[next_y][next_x] = pushing_color  # Placer la bille à sa nouvelle position
    
#     return True, f"Poussée effectuée. {billes_sorties} billes sorties du plateau.", billes_sorties


In [17]:
def push_marbles(board, pushing_coordinates, direction):
    """
    Effectue une poussée de billes si elle est valide
    """
    # Vérifier d'abord si la poussée est valide
    is_valid, message, pushed_coordinates = is_valid_push(board, pushing_coordinates, direction)
    if not is_valid:
        return False, message, 0
    
    # Sauvegarder les couleurs
    pushing_color = board[pushing_coordinates[0][1]][pushing_coordinates[0][0]]
    if pushed_coordinates:
        pushed_color = board[pushed_coordinates[0][1]][pushed_coordinates[0][0]]
    
    # Compter combien de billes vont sortir du plateau
    billes_sorties = 0
    for x, y in pushed_coordinates:
        neighbors = get_neighbors(board, x, y)
        if direction not in neighbors:
            billes_sorties += 1
    
    # Déplacer d'abord les billes poussées
    # On commence par la dernière bille pour éviter les conflits
    for x, y in reversed(pushed_coordinates):
        neighbors = get_neighbors(board, x, y)
        # Vérifier d'abord si la bille peut être déplacée
        if direction in neighbors:
            next_x, next_y = neighbors[direction]
            board[y][x] = 0  # Vider seulement si on peut déplacer
            board[next_y][next_x] = pushed_color
        else:
            board[y][x] = 0  # La bille sort du plateau
    
    # Ensuite déplacer les billes qui poussent
    # Là aussi, on commence par la dernière
    for i in range(len(pushing_coordinates)-1, -1, -1):
        x, y = pushing_coordinates[i]
        neighbors = get_neighbors(board, x, y)
        if direction in neighbors:  # Vérifier que la direction existe
            next_x, next_y = neighbors[direction]
            board[y][x] = 0
            board[next_y][next_x] = pushing_color
        else:
            return False, "Mouvement impossible pour les billes qui poussent", billes_sorties
    
    return True, f"Poussée effectuée. {billes_sorties} billes sorties du plateau.", billes_sorties

In [39]:
def push_marbles(board, pushing_coordinates, direction):
    """
    Effectue une poussée de billes si elle est valide
    """
    # Vérifier d'abord si la poussée est valide
    is_valid, message, pushed_coordinates = is_valid_push(board, pushing_coordinates, direction)
    if not is_valid:
        return False, message, 0
    
    # Sauvegarder les couleurs
    pushing_color = board[pushing_coordinates[0][1]][pushing_coordinates[0][0]]
    pushed_color = board[pushed_coordinates[0][1]][pushed_coordinates[0][0]]
    
    # Calculer les nouvelles positions pour toutes les billes
    new_positions = []
    for x, y in pushing_coordinates:
        neighbors = get_neighbors(board, x, y)
        if direction not in neighbors:
            return False, "Mouvement impossible pour les billes qui poussent", 0
        new_positions.append(neighbors[direction])
    
    # Calculer le nombre de billes qui vont sortir
    billes_sorties = 0
    for x, y in pushed_coordinates:
        neighbors = get_neighbors(board, x, y)
        if direction not in neighbors:
            billes_sorties += 1
            
    # Vider toutes les positions initiales
    for x, y in pushing_coordinates + pushed_coordinates:
        board[y][x] = 0
        
    # Placer les billes poussées qui ne sortent pas du plateau
    for i, (x, y) in enumerate(pushed_coordinates):
        neighbors = get_neighbors(board, x, y)
        if direction in neighbors:
            next_x, next_y = neighbors[direction]
            board[next_y][next_x] = pushed_color
            
    # Placer les billes qui poussent dans leurs nouvelles positions
    for x, y in new_positions:
        board[y][x] = pushing_color
    
    return True, f"Poussée effectuée. {billes_sorties} billes sorties du plateau.", billes_sorties

# Abalone Class

In [40]:
class AbaloneGame:
    def __init__(self):
        self.board = create_board()
        self.current_player = 'W'  # W commence toujours
        self.white_marbles_out = 0
        self.black_marbles_out = 0
    
    def switch_player(self):
        self.current_player = 'B' if self.current_player == 'W' else 'W'
    
    def get_current_player_name(self):
        return "Blanc" if self.current_player == 'W' else "Noir"
    
    def is_game_over(self):
        """Vérifie si un joueur a gagné"""
        if self.white_marbles_out >= 6:
            return True, "Les Noirs ont gagné!"
        if self.black_marbles_out >= 6:
            return True, "Les Blancs ont gagné!"
        return False, None
    
    def check_move_ownership(self, coordinates):
        """Vérifie si les billes appartiennent au joueur actuel"""
        for x, y in coordinates:
            if get_cell_content(self.board, x, y) != self.current_player:
                return False
        return True
    
    def make_move(self, coordinates, direction):
        """
        Effectue un mouvement (déplacement ou poussée)
        """
        # Vérifier si c'est game over
        is_over, message = self.is_game_over()
        if is_over:
            return False, message
        
        # Vérifier que les billes appartiennent au bon joueur
        if not self.check_move_ownership(coordinates):
            return False, "Ces billes ne vous appartiennent pas"
        
        # Essayer d'abord une poussée
        success, message, billes_sorties = push_marbles(self.board, coordinates, direction)
        if success:
            # Mettre à jour le compte des billes sorties
            if self.current_player == 'W':
                self.black_marbles_out += billes_sorties
            else:
                self.white_marbles_out += billes_sorties
            
            # Vérifier si quelqu'un a gagné
            is_over, win_message = self.is_game_over()
            if is_over:
                return True, f"{message} {win_message}"
            
            self.switch_player()
            return True, message
        
        # Si ce n'est pas une poussée valide, essayer un déplacement normal
        if len(coordinates) <= 3:
            success, message = make_move(self.board, coordinates, direction)
            if success:
                self.switch_player()
                return True, message
        
        return False, "Mouvement invalide"
    
    def display(self):
        """Affiche l'état actuel du jeu"""
        print(f"\nTour des {self.get_current_player_name()}s")
        print(f"Billes sorties - Blanches: {self.white_marbles_out}, Noires: {self.black_marbles_out}")
        display_board(self.board)


In [41]:
# Test de la classe
game = AbaloneGame()
print("Début de la partie!")
game.display()



Début de la partie!

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    W W W W W 
   W W W W W W 
  O O W W W O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O B B B O O 
   B B B B B B 
    B B B B B 


## Play game

In [42]:
def play_game():
    game = AbaloneGame()
    
    while True:
        game.display()
        
        print(f"\nTour des {game.get_current_player_name()}s")
        
        try:
            # Demander les coordonnées des billes SANS la direction
            coord_input = input("Entrez les coordonnées des billes (format: 'x1,y1 x2,y2 ...' ou 'q' pour quitter): ")
            
            if coord_input.lower() == 'q':
                print("Partie terminée!")
                break
            
            # Convertir l'entrée en liste de coordonnées
            coordinates = []
            coord_pairs = coord_input.strip().split()
            
            for coord in coord_pairs:
                x, y = map(int, coord.split(','))
                coordinates.append((x, y))
            
            # Demander la direction séparément
            direction = input("Entrez la direction (NW/NE/E/SE/SW/W): ").upper()
            if direction not in ['NW', 'NE', 'E', 'SE', 'SW', 'W']:
                print("Direction invalide!")
                continue
            
            # Effectuer le mouvement
            success, message = game.make_move(coordinates, direction)
            print(message)
            
            # Vérifier si la partie est terminée
            is_over, end_message = game.is_game_over()
            if is_over:
                game.display()
                print(end_message)
                break
                
        except ValueError:
            print("Format invalide! Utilisez: 'x1,y1 x2,y2' (ex: '2,3 2,4')")
        except IndexError:
            print("Coordonnées hors limites!")
        except Exception as e:
            print(f"Erreur: {str(e)}")

In [46]:
play_game()


Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    W W W W W 
   W W W W W W 
  O O W W W O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O B B B O O 
   B B B B B B 
    B B B B B 

Tour des Blancs

Début vérification poussée:
Coordonnées initiales: [(0, 0), (1, 1), (2, 2)]
Coordonnées après tri: [(0, 0), (1, 1), (2, 2)]
Vérification groupe: valide=True, message=Groupe valide, alignment=SE
Position à pousser: (3,3)
Vérification position (3,3): contenu=O
Billes poussées trouvées: []
Mouvement en ligne effectué avec succès

Tour des Noirs
Billes sorties - Blanches: 0, Noires: 0
    O W W W W 
   W W W W W W 
  O O W W W O O 
 O O O W O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O B B B O O 
   B B B B B B 
    B B B B B 

Tour des Noirs

Début vérification poussée:
Coordonnées initiales: [(4, 6), (4, 7), (4, 8)]
Coordonnées après tri: [(4, 8), (4, 7), (4, 6)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (4,5)
Vé

# Unit testing

## Push and win

In [43]:
def create_test_board(game, marble_positions):
    """
    Réinitialise le plateau du jeu avec des billes aux positions spécifiées
    Args:
        game: instance de AbaloneGame
        marble_positions: dict avec clés 'W' et 'B' contenant les listes de positions pour chaque couleur
    """
    # Vider d'abord le plateau
    for y in range(9):
        for x in range(9):
            if get_cell_content(game.board, x, y) in ['W', 'B']:
                game.board[y][x] = 0
                
    # Placer les billes selon les positions données
    for color in ['W', 'B']:
        if color in marble_positions:
            for x, y in marble_positions[color]:
                game.board[y][x] = 1 if color == 'W' else -1

def test_push_case_1():
    """Test de poussée: 3 blanches poussent 1 noire vers l'est"""
    game = AbaloneGame()
    positions = {
        'W': [(3,3), (4,3), (5,3)],
        'B': [(6,3)]
    }
    create_test_board(game, positions)
    
    print("Test poussée 3v1 vers l'est")
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,3), (4,3), (5,3)], 'E')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    game.display()
    
    # Vérification des positions finales
    expected_whites = [(4,3), (5,3), (6,3)]
    expected_black = [(7,3)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    # Vérifier la noire
    x, y = expected_black[0]
    if get_cell_content(game.board, x, y) != 'B':
        print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
        all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_push_case_2():
    """Test de poussée: 2 blanches poussent 1 noire vers l'est"""
    game = AbaloneGame()
    positions = {
        'W': [(4,3), (5,3)],
        'B': [(6,3)]
    }
    create_test_board(game, positions)
    
    print("Test poussée 2v1 vers l'est")
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    success, message = game.make_move([(4,3), (5,3)], 'E')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    game.display()
    
    # Vérifications
    expected_whites = [(5,3), (6,3)]
    expected_black = [(7,3)]
    
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    x, y = expected_black[0]
    if get_cell_content(game.board, x, y) != 'B':
        print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
        all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_push_case_3():
    """Test de poussée: 3 blanches poussent 1 noire vers l'est, avec une blanche derrière"""
    game = AbaloneGame()
    positions = {
        'W': [(2,3), (3,3), (4,3), (6,3)],
        'B': [(5,3)]
    }
    create_test_board(game, positions)
    
    print("Test poussée 2v1 vers l'est")
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    success, message = game.make_move([(2,3), (3,3), (4,3)], 'E')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    game.display()
    
    # Vérifications
    expected_whites = [(2,3), (3,3), (4,3), (6,3)]
    expected_black = [(5,3)]
    
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    x, y = expected_black[0]
    if get_cell_content(game.board, x, y) != 'B':
        print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
        all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_push_out_of_board():
    """Test de poussée: 2 blanches poussent 1 noire hors du plateau"""
    game = AbaloneGame()
    positions = {
        'W': [(5,5), (6,5)],  # Billes blanches sur le bord
        'B': [(7,5)]          # Bille noire qui sera poussée hors du plateau
    }
    create_test_board(game, positions)
    
    print("Test poussée sortant une bille du plateau")
    print("Configuration initiale:")
    print(f"Score actuel - Billes noires sorties: {game.black_marbles_out}")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(5,5), (6,5)], 'E')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    print(f"Score après poussée - Billes noires sorties: {game.black_marbles_out}")
    game.display()
    
    # Vérifications
    all_correct = True
    
    # Vérifier que les blanches ont bien bougé
    expected_whites = [(6,5), (7,5)]  # Nouvelles positions des blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    # Vérifier que le score a augmenté
    if game.black_marbles_out != 1:
        print(f"Erreur: attendu 1 bille noire sortie, trouvé {game.black_marbles_out}")
        all_correct = False
    
    # Vérifier que l'ancienne position est vide
    if get_cell_content(game.board, 5, 5) != 'O':
        print(f"Erreur: la position (5,5) devrait être vide")
        all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_win_condition():
    """Test de la condition de victoire: 6 billes sorties"""
    game = AbaloneGame()
    
    print("Test condition de victoire")
    print("État initial:")
    print(f"Billes noires sorties: {game.black_marbles_out}")
    
    # Simuler 5 billes déjà sorties
    game.white_marbles_out = 5
    # Configurer le plateau pour une dernière poussée
    positions = {
        'W': [(0,2)],  # Billes blanches sur le bord
        'B': [(2,4), (1,3)]          # Dernière bille noire qui sera poussée
    }
    create_test_board(game, positions)
    
    print("Configuration avant dernière poussée:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'B'
    
    # Effectuer la poussée finale
    success, message = game.make_move([(2,4), (1,3)], 'NW')
    
    print(f"Résultat: {message}")
    print(f"Configuration finale:")
    print(f"Billes blanches sorties: {game.white_marbles_out}")
    game.display()
    
    # Vérifications
    all_correct = True
    
    # Vérifier que le nombre de billes sorties est correct
    if game.white_marbles_out != 6:
        print(f"Erreur: attendu 6 billes blanches sorties, trouvé {game.white_marbles_out}")
        all_correct = False
    
    # Vérifier que la partie est terminée
    is_over, win_message = game.is_game_over()
    if not is_over:
        print("Erreur: la partie devrait être terminée")
        all_correct = False
    
    if "Noirs ont gagné" not in win_message:
        print(f"Erreur: message de victoire incorrect: {win_message}")
        all_correct = False
    
    # Essayer de faire un autre mouvement (devrait échouer)
    game.current_player = 'B'
    success, message = game.make_move([(1,1)], 'E')
    if success:
        print("Erreur: un mouvement a été accepté après la fin de la partie")
        all_correct = False
    
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_push_case_4():
    """Test de poussée non valide: 4 blanches poussent 2 noire vers l'est"""
    game = AbaloneGame()
    positions = {
        'W': [(1,3), (2,3), (3,3), (4,3)],
        'B': [(5,3),(6,3)]
    }
    create_test_board(game, positions)
    
    print("Test poussée 4v2 vers l'est")
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    success, message = game.make_move([(1,3), (2,3), (3,3), (4,3)], 'E')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    game.display()
    
    # Vérifications
    expected_whites = [(1,3), (2,3), (3,3), (4,3)]
    expected_black = [(5,3),(6,3)]
    
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    x, y = expected_black[0]
    if get_cell_content(game.board, x, y) != 'B':
        print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
        all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_push_case_5():
    """Test de poussée non valide: 3 noires poussent 1 blanche"""
    game = AbaloneGame()
    positions = {
        'W': [(1,0)],
        'B': [(2,1),(3,2),(4,3)]
    }
    create_test_board(game, positions)
    
    print("Test poussée 3v1 vers NW")
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des noires
    game.current_player = 'B'
    
    success, message = game.make_move([(2,1), (3,2), (4,3)], 'NW')
    
    print(f"Résultat: {message}")
    print("Configuration finale:")
    game.display()
    
    # Vérifications
    # expected_whites = []
    expected_black = [(1,0),(2,1),(3,2)]
    
    all_correct = True
    # for x, y in expected_whites:
    #     if get_cell_content(game.board, x, y) != 'W':
    #         print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
    #         all_correct = False
    for x, y in expected_black:
        if get_cell_content(game.board, x, y) != 'B':
            print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False   
    x, y = expected_black[0]
    # if get_cell_content(game.board, x, y) != 'B':
    #     print(f"Erreur: attendu B en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
    #     all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

In [44]:
test_push_case_5()

Test poussée 3v1 vers NW
Configuration initiale:

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    O W O O O 
   O O B O O O 
  O O O B O O O 
 O O O O B O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 

Début vérification poussée:
Coordonnées initiales: [(2, 1), (3, 2), (4, 3)]
Coordonnées après tri: [(4, 3), (3, 2), (2, 1)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (1,0)
Vérification position (1,0): contenu=W
Billes poussées trouvées: [(1, 0)]
Résultat: Poussée effectuée. 1 billes sorties du plateau.
Configuration finale:

Tour des Blancs
Billes sorties - Blanches: 1, Noires: 0
    O B O O O 
   O O B O O O 
  O O O B O O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 
Test réussi



True

In [23]:
test_push_case_5()

Test poussée 3v1 vers NW
Configuration initiale:

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    O W O O O 
   O O B O O O 
  O O O B O O O 
 O O O O B O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 

Début vérification poussée:
Coordonnées initiales: [(2, 1), (3, 2), (4, 3)]
Coordonnées après tri: [(4, 3), (3, 2), (2, 1)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (1,0)
Vérification position (1,0): contenu=W
Billes poussées trouvées: [(1, 0)]
Résultat: Poussée effectuée. 1 billes sorties du plateau.
Configuration finale:

Tour des Blancs
Billes sorties - Blanches: 1, Noires: 0
    O B O O O 
   O O O O O O 
  O O O O O O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 
Erreur: attendu B en 2,1, trouvé O
Erreur: attendu B en 3,2, trouvé O
Test échoué



False

In [45]:
test_push_out_of_board()

Test poussée sortant une bille du plateau
Configuration initiale:
Score actuel - Billes noires sorties: 0

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O W W B 
  O O O O O O O 
   O O O O O O 
    O O O O O 

Début vérification poussée:
Coordonnées initiales: [(5, 5), (6, 5)]
Coordonnées après tri: [(5, 5), (6, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Position à pousser: (7,5)
Vérification position (7,5): contenu=B
Billes poussées trouvées: [(7, 5)]
Résultat: Poussée effectuée. 1 billes sorties du plateau.
Configuration finale:
Score après poussée - Billes noires sorties: 1

Tour des Noirs
Billes sorties - Blanches: 0, Noires: 1
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O O O O O O 
O O O O O O O O O 
 O O O O O O W W 
  O O O O O O O 
   O O O O O O 
    O O O O O 
Test réussi



True

## Movement upper

In [25]:
def test_para_case_1():
    """Cas 1 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    print("Configuration initiale:")
    game.display()
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,1), (3,2)], 'NW')
    
    print(f"Résultat: {message}")
    
    # Vérification des positions finales
    expected_whites = [(2,0), (2,1)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_para_case_2():
    """Cas 2 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,1), (3,2)], 'E')
    
    print(f"Résultat: {message}")
    
    # Vérification des positions finales
    expected_whites = [(4,1), (4,2)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_para_case_3():
    """Cas 3 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,1), (3,2)], 'SE')
    
    print(f"Résultat: {message}")

    
    # Vérification des positions finales
    expected_whites = [(4,2), (4,3)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_para_case_4():
    """Cas 4 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,1), (3,2)], 'W')
    
    print(f"Résultat: {message}")
    
    # Vérification des positions finales
    expected_whites = [(2,1), (2,2)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
            
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_para_case_5():
    """Cas 5 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    
    # Forcer le tour des blancs
    game.current_player = 'W'
    
    # Test du mouvement
    success, message = game.make_move([(3,1), (3,2)], 'NE')
    
    print(f"Résultat: {message}")
    # Vérification des positions finales
    expected_whites = [(3,0), (3,1)]
    
    all_correct = True
    # Vérifier les blanches
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct

def test_para_case_6():
    """Cas 6 des mouv parallèles dans la partie supérieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,1), (3,2)],
        'B': []
    }
    create_test_board(game, positions)
    
    game.current_player = 'W'
    
    success, message = game.make_move([(3,1), (3,2)], 'SW')
    
    print(f"Résultat: {message}")
    game.display()
    
    expected_whites = [(3,2), (3,3)]
    
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            print(f"Erreur: attendu W en {x},{y}, trouvé {get_cell_content(game.board, x, y)}")
            all_correct = False
        
    print(f"Test {'réussi' if all_correct else 'échoué'}\n")
    return all_correct


## Movement lower

In [26]:
def test_para_case_lower_1():
    """Test mouvement parallèle NW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'NW')
    
    expected_whites = [(3,5), (3,6)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_lower_2():
    """Test mouvement parallèle E dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'E')
    
    expected_whites = [(4,6), (4,7)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_lower_3():
    """Test mouvement parallèle SE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'SE')
    
    expected_whites = [(3,7), (3,8)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
         display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_lower_4():
    """Test mouvement parallèle W dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'W')
    
    expected_whites = [(2,6), (2,7)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_lower_5():
    """Test mouvement parallèle NE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'NE')
    
    expected_whites = [(4,5), (4,6)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_lower_6():
    """Test mouvement parallèle SW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,6), (3,7)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,6), (3,7)], 'SW')
    
    expected_whites = [(2,7), (2,8)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)
    return all_correct

## Movement Mid board

In [27]:
def test_para_case_mid_1():
    """Test mouvement parallèle NW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'NW')
    
    expected_whites = [(3,3), (4,3)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_mid_2():
    """Test mouvement parallèle E dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'E')
    
    expected_whites = [(6,4), (5,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_mid_3():
    """Test mouvement parallèle SE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'SE')
    
    expected_whites = [(4,5), (5,5)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
         display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_mid_4():
    """Test mouvement parallèle W dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'W')
    
    expected_whites = [(3,4), (4,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_mid_5():
    """Test mouvement parallèle NE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'NE')
    
    expected_whites = [(4,3), (5,3)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_para_case_mid_6():
    """Test mouvement parallèle SW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(4,4), (5,4)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(4,4), (5,4)], 'SW')
    
    expected_whites = [(3,5), (4,5)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)
    return all_correct

In [28]:
def test_verti_case_mid_1():
    """Test mouvement parallèle NW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'NW')
    
    expected_whites = [(3,4), (3,3), (3,2)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_verti_case_mid_2():
    """Test mouvement parallèle E dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'E')
    
    expected_whites = [(4,5), (5,4), (5,3)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_verti_case_mid_3():
    """Test mouvement parallèle SE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'SE')
    
    expected_whites = [(3,6), (4,5), (5,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
         display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_verti_case_mid_4():
    """Test mouvement parallèle W dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'W')
    
    expected_whites = [(2,5), (3,4), (3,3)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_verti_case_mid_5():
    """Test mouvement parallèle NE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'NE')
    
    expected_whites = [(4,2), (4,3), (4,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_verti_case_mid_6():
    """Test mouvement parallèle SW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(3,5), (4,4), (4,3)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(3,5), (4,4), (4,3)], 'SW')
    
    expected_whites = [(2,6), (3,5), (4,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)
    return all_correct

In [29]:
def test_hori_bot_1():
    """Test mouvement parallèle NW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'NW')
    
    expected_whites = [(1,4), (2,4), (3,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_hori_bot_2():
    """Test mouvement parallèle E dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'E')
    
    expected_whites = [(2,5), (3,5), (4,5)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_hori_bot_3():
    """Test mouvement parallèle SE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'SE')
    
    expected_whites = [(1,6), (2,6), (3,6)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
         display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_hori_bot_4():
    """Test mouvement parallèle W dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'W')
    
    expected_whites = [(0,5), (1,5), (2,5)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)

    return all_correct

def test_hori_bot_5():
    """Test mouvement parallèle NE dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'NE')
    
    expected_whites = [(2,4), (3,4), (4,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

def test_hori_bot_6():
    """Test mouvement parallèle SW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [(1,5), (2,5), (3,5)],
        'B': []
    }
    create_test_board(game, positions)
    game.current_player = 'W'
    success, message = game.make_move([(1,5), (2,5), (3,5)], 'SW')
    
    expected_whites = [(0,6), (1,6), (2,6)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'W':
            all_correct = False
    print("Test réussi",all_correct)
    return all_correct

In [30]:
def test_hori_bot():
    test_hori_bot_1()
    test_hori_bot_2()
    test_hori_bot_3()
    test_hori_bot_4()
    test_hori_bot_5()
    test_hori_bot_6()

In [31]:
def test_bot_to_top():
    """Test mouvement parallèle NW dans la partie inférieure"""
    game = AbaloneGame()
    positions = {
        'W': [],
        'B': [(1,7), (1,8)]
    }
    create_test_board(game, positions)
    game.current_player = 'B'
    success, message = game.make_move([(1,7), (1,8)], 'NW')
    game.current_player = 'B'
    success, message = game.make_move([(1,6), (1,7)], 'NW')
    game.current_player = 'B'
    success, message = game.make_move([(1,5), (1,6)], 'NW')
    game.current_player = 'B'
    success, message = game.make_move([(1,4), (1,5)], 'NW')
    #success, message = game.make_move([(1,7), (1,8)], 'NW')
    expected_whites = [(0,3), (1,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'B':
            all_correct = False
    success, message = game.make_move([(0,3), (1,4)], 'NW')
    expected_whites = [(0,3), (1,4)]
    all_correct = True
    for x, y in expected_whites:
        if get_cell_content(game.board, x, y) != 'B':
            all_correct = False
    if all_correct == False:
        display_board(game.board)
    print("Test réussi",all_correct)

    return all_correct

In [32]:
test_bot_to_top()


Début vérification poussée:
Coordonnées initiales: [(1, 7), (1, 8)]
Coordonnées après tri: [(1, 8), (1, 7)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (1,6)
Vérification position (1,6): contenu=O
Billes poussées trouvées: []

Début vérification poussée:
Coordonnées initiales: [(1, 6), (1, 7)]
Coordonnées après tri: [(1, 7), (1, 6)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (1,5)
Vérification position (1,5): contenu=O
Billes poussées trouvées: []

Début vérification poussée:
Coordonnées initiales: [(1, 5), (1, 6)]
Coordonnées après tri: [(1, 6), (1, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Position à pousser: (1,4)
Vérification position (1,4): contenu=O
Billes poussées trouvées: []

Début vérification poussée:
Coordonnées initiales: [(1, 4), (1, 5)]
Coordonnées après tri: [(1, 5), (1, 4)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NW
Po

True

In [33]:
test_hori_bot()


Début vérification poussée:
Coordonnées initiales: [(1, 5), (2, 5), (3, 5)]
Coordonnées après tri: [(1, 5), (2, 5), (3, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Test réussi True

Début vérification poussée:
Coordonnées initiales: [(1, 5), (2, 5), (3, 5)]
Coordonnées après tri: [(1, 5), (2, 5), (3, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Position à pousser: (4,5)
Vérification position (4,5): contenu=O
Billes poussées trouvées: []
Test réussi True

Début vérification poussée:
Coordonnées initiales: [(1, 5), (2, 5), (3, 5)]
Coordonnées après tri: [(1, 5), (2, 5), (3, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Test réussi True

Début vérification poussée:
Coordonnées initiales: [(1, 5), (2, 5), (3, 5)]
Coordonnées après tri: [(3, 5), (2, 5), (1, 5)]
Vérification groupe: valide=True, message=Groupe valide, alignment=W
Position à pousser: (0,5)
Vérification position (0,5): contenu=O
Billes poussée

# Run all test

In [34]:
def run_all_tests():
    """Exécute tous les tests et affiche un résumé"""
    tests = [
        test_push_case_1,
        test_push_case_2,
        test_push_case_3,
        test_push_case_4,
        test_push_out_of_board,
        # test_para_case_1,
        # test_para_case_2,
        # test_para_case_3,
        # test_para_case_4,
        # test_para_case_5,
        # test_para_case_6,
        # test_para_case_lower_1,
        # test_para_case_lower_2,
        # test_para_case_lower_3,
        # test_para_case_lower_4,
        # test_para_case_lower_5,
        # test_para_case_lower_6,
        # test_para_case_mid_1,
        # test_para_case_mid_2,
        # test_para_case_mid_3,
        # test_para_case_mid_4,
        # test_para_case_mid_5,
        # test_para_case_mid_6,
        # test_verti_case_mid_1,
        # test_verti_case_mid_2,
        # test_verti_case_mid_3,
        # test_verti_case_mid_4,
        # test_verti_case_mid_5,
        # test_verti_case_mid_6
        test_win_condition
    ]
    
    total = len(tests)
    passed = 0
    
    print("Début des tests...")
    print("="*50)
    
    for test in tests:
        if test():
            passed += 1
            
    print("="*50)
    print(f"Résultats: {passed}/{total} tests réussis")


## Test lower3

In [35]:
test_verti_case_mid_1()


Début vérification poussée:
Coordonnées initiales: [(3, 5), (4, 4), (4, 3)]
Coordonnées après tri: [(3, 5), (4, 4), (4, 3)]
Vérification groupe: valide=True, message=Groupe valide, alignment=NE
Test réussi True


True

In [36]:
test_para_case_lower_3()


Début vérification poussée:
Coordonnées initiales: [(3, 6), (3, 7)]
Coordonnées après tri: [(3, 6), (3, 7)]
Vérification groupe: valide=True, message=Groupe valide, alignment=SE
Position à pousser: (3,8)
Vérification position (3,8): contenu=O
Billes poussées trouvées: []
Test réussi True


True

In [37]:
run_all_tests()

Début des tests...
Test poussée 3v1 vers l'est
Configuration initiale:

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O W W W B O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 

Début vérification poussée:
Coordonnées initiales: [(3, 3), (4, 3), (5, 3)]
Coordonnées après tri: [(3, 3), (4, 3), (5, 3)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Position à pousser: (6,3)
Vérification position (6,3): contenu=B
Vérification position (7,3): contenu=O
Billes poussées trouvées: [(6, 3)]
Résultat: Poussée effectuée. 0 billes sorties du plateau.
Configuration finale:

Tour des Noirs
Billes sorties - Blanches: 0, Noires: 0
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O O W W W B 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 
Test réussi

Test poussée 2v1 vers l'est
Configuration initiale:

Tour des Blancs
Billes sorties - Bl

In [38]:
run_all_tests()

Début des tests...
Test poussée 3v1 vers l'est
Configuration initiale:

Tour des Blancs
Billes sorties - Blanches: 0, Noires: 0
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O W W W B O 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 

Début vérification poussée:
Coordonnées initiales: [(3, 3), (4, 3), (5, 3)]
Coordonnées après tri: [(3, 3), (4, 3), (5, 3)]
Vérification groupe: valide=True, message=Groupe valide, alignment=E
Position à pousser: (6,3)
Vérification position (6,3): contenu=B
Vérification position (7,3): contenu=O
Billes poussées trouvées: [(6, 3)]
Résultat: Poussée effectuée. 0 billes sorties du plateau.
Configuration finale:

Tour des Noirs
Billes sorties - Blanches: 0, Noires: 0
    O O O O O 
   O O O O O O 
  O O O O O O O 
 O O O O W W W B 
O O O O O O O O O 
 O O O O O O O O 
  O O O O O O O 
   O O O O O O 
    O O O O O 
Test réussi

Test poussée 2v1 vers l'est
Configuration initiale:

Tour des Blancs
Billes sorties - Bl